In [113]:
from tensorflow.keras.datasets import imdb 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Embedding, Flatten, LSTM
from tensorflow.keras import utils 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline

In [114]:
start = '<review_text>'
end = '</review_text>'
with open('all.txt', 'r') as f:
    texts_true = f.read(100000)

#print(texts_true)
import re
result = ' '.join(re.findall(r'<review_text>([^<>]+)</review_text>', texts_true))
#print(result)
result = result.replace('\n', ' ')
print(result)
#print( ' '.join(re.findall(r'<review_text>([^<>]+)</review_text>', texts_true)))

 This book has its good points. If anything, it helps you put into words what you want  from a supervisor, but it is not very accurate. The online test does not account for a difference between when 2 of their options are both exactly like you, or if they don't describe you at all. This messes up the results, and it did not describe me very well. I am not just in denial. I have taken a lot of personality type tests, like the Keirsey Temperment sorter and have a pretty good idea of my strengths. So, although this book is pretty good in making you understand the importance of incouraging your strengths, it still leaves you wondering about how you fit in to their terminology.  As for using this book as a manager to better serve your employees, I'v seen it done and it does not necessarily work because the strengths spit out for people were not wholly accurate. The company I work for has done this, and most of the people who were shifted around to better serve their strengths (according to 

In [115]:
count_true = len(result)
print("count = ",count_true)

count =  51806


In [116]:
from keras.preprocessing.text import Tokenizer
# разбиваем эти высказывания на слова
maxWordsCount = 20000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                         lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(result)

word_2_index = tokenizer.word_index
word_2_index.items()

dict_items([('e', 1), ('t', 2), ('o', 3), ('i', 4), ('n', 5), ('s', 6), ('r', 7), ('h', 8), ('l', 9), ('d', 10), ('u', 11), ('c', 12), ('g', 13), ('w', 14), ('f', 15), ('b', 16), ('y', 17), ('v', 18), ('k', 19), ("'", 20), ('j', 21), ('x', 22), ('0', 23), ('q', 24), ('z', 25), ('1', 26), ('2', 27), ('6', 28), ('9', 29), ('3', 30), ('7', 31), ('4', 32), ('8', 33), ('5', 34)])

In [117]:
#начальный список с частотами появления
dist = list(tokenizer.word_counts.items())
print(dist[:10])
print(result[0][:100])

[('t', 4021), ('h', 2180), ('i', 3163), ('s', 2663), ('b', 767), ('o', 3448), ('k', 444), ('g', 873), ('d', 1480), ('n', 2786)]
 


In [118]:
#преобразуем текст в последовательность чисел в соответствии с полученным словарем
data = tokenizer.texts_to_sequences(result)

In [119]:
max_text_len = 190
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)
print(data_pad.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 2]
 [0 0 0 ... 0 0 8]
 ...
 [0 0 0 ... 0 0 5]
 [0 0 0 ... 0 0 2]
 [0 0 0 ... 0 0 0]]
(51806, 190)


In [120]:
X = data_pad
Y = np.array([[1, 0]]*count_true)
print(X.shape, Y.shape)

indeces = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indeces]
Y = Y[indeces]
#print(X, Y)

(51806, 190) (51806, 2)


In [121]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.2)

X_train.shape, Y_train.shape

((41444, 190), (41444, 2))

In [122]:
from tensorflow.keras.optimizers import Adam
#рекуррентный слой LSTM
model = Sequential()
model.add(Embedding(maxWordsCount, 128, input_length = max_text_len))
model.add(LSTM(64, activation='tanh', return_sequences=True))
model.add(LSTM(32, activation='tanh'))
model.add(Dense(2, activation='softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

history = model.fit(X_train, Y_train, batch_size=32, epochs=25, validation_data=(X_test, Y_test))

scores = model.evaluate(X_test, Y_test, verbose=1)
model.save('en-de-model.h5')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 190, 128)          2560000   
                                                                 
 lstm_6 (LSTM)               (None, 190, 64)           49408     
                                                                 
 lstm_7 (LSTM)               (None, 32)                12416     
                                                                 
 dense_3 (Dense)             (None, 2)                 66        
                                                                 
Total params: 2,621,890
Trainable params: 2,621,890
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
1296/1296 [==============================] - 138s 105ms/step - loss: 0.0229 - accuracy: 1.0000 - val_loss: 1.6491e-04 - val_accuracy: 1.0000
Epoch 2/25
1296/1296 [==

t = "It's a good book".lower()
data = tokenizer.texts_to_sequences([t])
data_pad = pad_sequences(data, maxlen=max_text_len)

reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
 
def sequence_to_text(list_of_indices):
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    
print(sequence_to_text(data[0]))

print(data)

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

def vectorize_sequences(sequences, dimension=10000): 
    results = np.zeros((len(sequences), dimension)) 
    for i, sequence in enumerate(sequences): 
        results[i, sequence] = 1. 
        return results

x_train = vectorize_sequences(x_train, max_words) 
x_test = vectorize_sequences(x_test, max_words) 

In [123]:
embedding_matrix = model.layers[0].get_weights()[0]

In [124]:
word = 'good'

data = tokenizer.texts_to_sequences(word)
data_pad = pad_sequences(data, maxlen=max_text_len)
print('Номер слова', data) 
#print('Вектор для слова', data_pad)
model.predict(data_pad, batch_size= 5, verbose=1)

Номер слова [[13], [3], [3], [10]]
1/1 [==============================] - 1s 557ms/step


array([[1.0000000e+00, 3.0770495e-09],
       [1.0000000e+00, 3.0765741e-09],
       [1.0000000e+00, 3.0765741e-09],
       [1.0000000e+00, 3.0767502e-09]], dtype=float32)

##Получается, что данный комментарий с вероятностью 0.9999993 является хорошим, а с 0.0000007534602 - плохим

In [125]:
# Creating a reverse dictionary
reverse_word_map = word_2_index.items()

# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

t = 'I like this movie. Great acting and an interesting plot'
data = tokenizer.texts_to_sequences([t])
data_pad = pad_sequences(data, maxlen=max_text_len)
res = model.predict(data_pad)
print(res, np.argmax(res), sep='\n')

1/1 [==============================] - 0s 30ms/step
[[1.0000000e+00 3.0789693e-09]]
0


##Получается, что данный комментарий с вероятностью 0.53 является хорошим, а с 0.46 - плохим

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()